<h1>Building Dataset From Riot API</h1>

In [1]:
import json
import requests
import warnings
import time
import pickle

In [58]:
file = open("riot_api.json", 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

In [30]:
def getMatchesByRank(rank, subrank):
    request_url = f"https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{subrank}?page=1&api_key={credentials["key"]}"
    resp = requests.get(request_url)
    summoner_ids = []
    if (resp.status_code == 200):
        print(resp.json()[0])
        for summoner in resp.json():
            summoner_ids.append(summoner["summonerId"])
    else:
        warnings.warn(f"Error: Response Code {resp.status_code}")
    
    # API LIMITS
    # 20 requests every second
    # 100 requests every 120 seconds
    short_request_count = 1
    long_request_count = 3
    short_request_max = 20
    long_request_max = 100
    short_request_cooldown = 1
    long_request_cooldown = 120
    short_request_start_time = time.perf_counter()
    long_request_start_time = time.perf_counter()
    def check_requests(short_request_count, long_request_count, short_request_start_time, long_request_start_time, short_request_max, long_request_max, short_request_cooldown, long_request_cooldown):
        if time.perf_counter() - long_request_start_time < long_request_cooldown and long_request_count >= long_request_max:
            print(f"Sleeping for: {long_request_cooldown - (time.perf_counter() - long_request_start_time):.2f} seconds")
            time.sleep(long_request_cooldown - (time.perf_counter() - long_request_start_time))
            long_request_start_time = time.perf_counter()
            short_request_start_time = time.perf_counter()
            long_request_count = 1
            short_request_count = 1
            return (short_request_count, long_request_count, short_request_start_time, long_request_start_time)
        elif time.perf_counter() - long_request_start_time > long_request_cooldown:
            print("Restarting long request")
            long_request_start_time = time.perf_counter()
            long_request_count = 1
            return (short_request_count, long_request_count, short_request_start_time, long_request_start_time)
        elif time.perf_counter() - short_request_start_time < short_request_cooldown and short_request_count >= short_request_max:
            print(f"Sleeping for: {short_request_cooldown - (time.perf_counter() - short_request_start_time):.2f} seconds")
            time.sleep(short_request_cooldown - (time.perf_counter() - short_request_start_time))
            short_request_start_time = time.perf_counter()
            short_request_count = 1
            return (short_request_count, long_request_count, short_request_start_time, long_request_start_time)
        elif time.perf_counter() - short_request_start_time > short_request_cooldown:
            print("Restarting short request")
            short_request_start_time = time.perf_counter()
            short_request_count = 1
            return (short_request_count, long_request_count, short_request_start_time, long_request_start_time)
        return (short_request_count, long_request_count, short_request_start_time, long_request_start_time)
    
    total_match_ids = []
    
    for summoner_id in summoner_ids:
        get_summoner_request_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={credentials["key"]}"
        resp = requests.get(get_summoner_request_url)
        puuid = None
        if (resp.status_code == 200):
            puuid = resp.json()["puuid"]
        else:
            warnings.warn(f"Error: Response Code {resp.status_code}")
            if resp.status_code == 429:
                print("Rate limit exceeded. Sleeping for 120 seconds")
                time.sleep(120)
                request_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={credentials["key"]}"
                resp = requests.get(request_url)
                if (resp.status_code == 200):
                    puuid = resp.json()["puuid"]
                short_request_count = 1
                long_request_count = 1
                short_request_time = time.perf_counter()
                long_request_time = short_request_time
        
        short_request_count += 1
        long_request_count += 1
        print(f"Short request count: {short_request_count}")
        print(f"Long request count: {long_request_count}")
        short_request_count, long_request_count, short_request_start_time, long_request_start_time = check_requests(short_request_count, long_request_count, short_request_start_time, long_request_start_time, short_request_max, long_request_max, short_request_cooldown, long_request_cooldown)
        if puuid != None:
            get_matches_request_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10&api_key={credentials["key"]}"
            resp = requests.get(get_matches_request_url)
            if (resp.status_code == 200):
                for match in resp.json():
                    total_match_ids.append(match)
            else:
                warnings.warn(f"Error: Response Code {resp.status_code}")
                if resp.status_code == 429:
                    print("Rate limit exceeded. Sleeping for 120 seconds")
                    time.sleep(120)
                    get_matches_request_url = get_matches_request_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10&api_key={credentials["key"]}"
                    resp = requests.get(get_matches_request_url)
                    if (resp.status_code == 200):
                        for match in resp.json():
                            total_match_ids.append(match)
                    short_request_count = 1
                    long_request_count = 1
                    short_request_time = time.perf_counter()
                    long_request_time = short_request_time
            
            short_request_count += 1
            long_request_count += 1
            print(f"Short request count: {short_request_count}")
            print(f"Long request count: {long_request_count}")
            short_request_count, long_request_count, short_request_start_time, long_request_start_time = check_requests(short_request_count, long_request_count, short_request_start_time, long_request_start_time, short_request_max, long_request_max, short_request_cooldown, long_request_cooldown)    
    print("********************FINISHED**********************")
    return total_match_ids

In [3]:
file_path = 'data.pickle'

# with open(file_path, 'wb') as file:
#     pickle.dump(total_match_ids, file)

In [4]:
# iron4_matches = None

# with open(file_path, 'rb') as file:
#     iron4_matches = pickle.load(file)

In [19]:
iron3_matches = getMatchesByRank("IRON", "III")

{'leagueId': '5fb61387-cc1e-463f-9ae2-49a11e84faf4', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'IRON', 'rank': 'III', 'summonerId': '0IwuPXtx01x2pcAFijnz-ymYktfOkHXgypaeFrjmxAQ5p7dI', 'summonerName': 'Crazyknees', 'leaguePoints': 37, 'wins': 30, 'losses': 35, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 4
Long request count: 6
Restarting short request
Short request count: 5
Long request count: 7
Restarting short request
Short request count: 6
Long request count: 8
Restarting short request


C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\1838558130.py:82: UserWarning: Error: Response Code 500
  warnings.warn(f"Error: Response Code {resp.status_code}")


Short request count: 7
Long request count: 9
Restarting short request
Short request count: 8
Long request count: 10
Restarting short request
Short request count: 9
Long request count: 11
Restarting short request
Short request count: 10
Long request count: 12
Restarting short request
Short request count: 11
Long request count: 13
Restarting short request
Short request count: 12
Long request count: 14
Restarting short request
Short request count: 13
Long request count: 15
Restarting short request
Short request count: 14
Long request count: 16
Restarting short request
Short request count: 15
Long request count: 17
Restarting short request
Short request count: 16
Long request count: 18
Restarting short request
Short request count: 17
Long request count: 19
Restarting short request
Short request count: 18
Long request count: 20
Restarting short request
Short request count: 19
Long request count: 21
Restarting short request
Short request count: 20
Long request count: 22
Restarting short requ

C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\1838558130.py:82: UserWarning: Error: Response Code 503
  warnings.warn(f"Error: Response Code {resp.status_code}")


Short request count: 99
Long request count: 101
Restarting long request
Short request count: 100
Long request count: 102
Restarting long request
Short request count: 101
Long request count: 103
Restarting long request
Short request count: 102
Long request count: 104
Restarting long request
Short request count: 103
Long request count: 105
Restarting long request
Short request count: 104
Long request count: 106
Restarting long request
Short request count: 105
Long request count: 107
Restarting long request
Short request count: 106
Long request count: 108
Restarting long request
Short request count: 107
Long request count: 109
Restarting long request
Short request count: 108
Long request count: 110
Restarting long request
Short request count: 109
Long request count: 111
Restarting long request
Short request count: 110
Long request count: 112
Restarting long request
Short request count: 111
Long request count: 113
Restarting long request
Short request count: 112
Long request count: 114
Res

In [21]:
file_path = 'iron3_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(iron3_matches, file)

In [22]:
iron2_matches = getMatchesByRank("IRON", "II")

{'leagueId': '3a4577df-0334-4fb4-89c7-e1044f2ce6fd', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'IRON', 'rank': 'II', 'summonerId': 'E4YIS0_mEt6L1rFs0Rqu1_vzxW54ZbTMM85T8H3JlSztWq7V', 'summonerName': 'Davidc409', 'leaguePoints': 54, 'wins': 3, 'losses': 12, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 4
Long request count: 6
Restarting short request
Short request count: 5
Long request count: 7
Restarting short request
Short request count: 6
Long request count: 8
Restarting short request
Short request count: 7
Long request count: 9
Restarting short request
Short request count: 8
Long request count: 10
Restarting short request
Short request count: 9
Long request count: 11
Restarting short request
Short request count: 10
Long request count: 12
Restarting short request
Short request count: 11
Long request count: 13
Restarting 

C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\1838558130.py:56: UserWarning: Error: Response Code 429
  warnings.warn(f"Error: Response Code {resp.status_code}")


Rate limit exceeded. Sleeping for 120 seconds
Short request count: 2
Long request count: 2
Restarting long request
Short request count: 3
Long request count: 3
Restarting long request
Short request count: 4
Long request count: 4
Restarting long request
Short request count: 5
Long request count: 5
Restarting long request
Short request count: 6
Long request count: 6
Restarting long request
Short request count: 7
Long request count: 7
Restarting long request
Short request count: 8
Long request count: 8
Restarting long request
Short request count: 9
Long request count: 9
Restarting long request
Short request count: 10
Long request count: 10
Restarting long request
Short request count: 11
Long request count: 11
Restarting long request
Short request count: 12
Long request count: 12
Restarting long request
Short request count: 13
Long request count: 13
Restarting long request
Short request count: 14
Long request count: 14
Restarting long request
Short request count: 15
Long request count: 15


In [26]:
file_path = 'iron2_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(iron2_matches, file)

In [25]:
iron1_matches = getMatchesByRank("IRON", "I")

{'leagueId': '583c0bdb-e147-405e-a2db-44c05c3afd02', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'IRON', 'rank': 'I', 'summonerId': 'PN2ZA37AVur52HydxPDpmbRNPrpZ1ulVUE1z2r_eP8COnvnR', 'summonerName': 'HakuTnoface', 'leaguePoints': 88, 'wins': 3, 'losses': 2, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 4
Long request count: 6
Restarting short request
Short request count: 5
Long request count: 7
Restarting short request
Short request count: 6
Long request count: 8
Restarting short request
Short request count: 7
Long request count: 9
Restarting short request
Short request count: 8
Long request count: 10
Restarting short request
Short request count: 9
Long request count: 11
Restarting short request
Short request count: 10
Long request count: 12
Restarting short request
Short request count: 11
Long request count: 13
Restarting 

C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\1838558130.py:56: UserWarning: Error: Response Code 429
  warnings.warn(f"Error: Response Code {resp.status_code}")


Rate limit exceeded. Sleeping for 120 seconds
Short request count: 2
Long request count: 2
Restarting long request
Short request count: 3
Long request count: 3
Restarting long request
Short request count: 4
Long request count: 4
Restarting long request
Short request count: 5
Long request count: 5
Restarting long request
Short request count: 6
Long request count: 6
Restarting long request
Short request count: 7
Long request count: 7
Restarting long request
Short request count: 8
Long request count: 8
Restarting long request
Short request count: 9
Long request count: 9
Restarting long request
Short request count: 10
Long request count: 10
Restarting long request
Short request count: 11
Long request count: 11
Restarting long request
Short request count: 12
Long request count: 12
Restarting long request
Short request count: 13
Long request count: 13
Restarting long request
Short request count: 14
Long request count: 14
Restarting long request
Short request count: 15
Long request count: 15


In [27]:
file_path = 'iron1_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(iron1_matches, file)

In [28]:
bronze4_matches = getMatchesByRank("BRONZE", "IV")

{'leagueId': '67978e41-b158-4795-84e7-b238626ec08d', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'BRONZE', 'rank': 'IV', 'summonerId': 'DY5PFwyf1GIO14KzJdzRYFZS0y_5iJNkJWhRZj7Oo6M6miDj', 'summonerName': 'AnnNya', 'leaguePoints': 21, 'wins': 5, 'losses': 3, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 4
Long request count: 6
Restarting short request
Short request count: 5
Long request count: 7
Restarting short request
Short request count: 6
Long request count: 8
Restarting short request
Short request count: 7
Long request count: 9
Restarting short request
Short request count: 8
Long request count: 10
Restarting short request
Short request count: 9
Long request count: 11
Restarting short request
Short request count: 10
Long request count: 12
Restarting short request
Short request count: 11
Long request count: 13
Restarting sho

C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\1838558130.py:56: UserWarning: Error: Response Code 429
  warnings.warn(f"Error: Response Code {resp.status_code}")


Rate limit exceeded. Sleeping for 120 seconds
Short request count: 2
Long request count: 2
Restarting long request
Short request count: 3
Long request count: 3
Restarting long request
Short request count: 4
Long request count: 4
Restarting long request
Short request count: 5
Long request count: 5
Restarting long request
Short request count: 6
Long request count: 6
Restarting long request
Short request count: 7
Long request count: 7
Restarting long request
Short request count: 8
Long request count: 8
Restarting long request
Short request count: 9
Long request count: 9
Restarting long request
Short request count: 10
Long request count: 10
Restarting long request
Short request count: 11
Long request count: 11
Restarting long request
Short request count: 12
Long request count: 12
Restarting long request
Short request count: 13
Long request count: 13
Restarting long request
Short request count: 14
Long request count: 14
Restarting long request
Short request count: 15
Long request count: 15


In [31]:
file_path = 'bronze4_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(bronze4_matches, file)

In [32]:
bronze3_matches = getMatchesByRank("BRONZE", "III")

{'leagueId': '9dc91b6d-5066-45c0-a9e2-13a52c39f1e8', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'BRONZE', 'rank': 'III', 'summonerId': '1ObaU_c3b6Qqazws2kWS6EZWoblh9pOG4bzvKLc9ngMt3oqE', 'summonerName': 'GrannyLover', 'leaguePoints': 64, 'wins': 4, 'losses': 5, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 2
Long request count: 6
Short request count: 3
Long request count: 7
Restarting short request
Short request count: 2
Long request count: 8
Short request count: 3
Long request count: 9
Short request count: 4
Long request count: 10
Restarting short request
Short request count: 2
Long request count: 11
Short request count: 3
Long request count: 12
Restarting short request
Short request count: 2
Long request count: 13
Short request count: 3
Long request count: 14
Restarting short request
Short request count: 2
Long request co

In [33]:
file_path = 'bronze3_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(bronze3_matches, file)

In [34]:
bronze2_matches = getMatchesByRank("BRONZE", "II")

{'leagueId': 'e44facda-db8a-4202-b017-90fc78b8dd5a', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'BRONZE', 'rank': 'II', 'summonerId': 'a-C92ihtk8eD8B9FM4SoMDcdcmrxEGqhUQc8e_8dOyATsw7F', 'summonerName': 'DieCatsKing', 'leaguePoints': 16, 'wins': 15, 'losses': 16, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 2
Long request count: 6
Short request count: 3
Long request count: 7
Restarting short request
Short request count: 2
Long request count: 8
Short request count: 3
Long request count: 9
Restarting short request
Short request count: 2
Long request count: 10
Short request count: 3
Long request count: 11
Restarting short request
Short request count: 2
Long request count: 12
Short request count: 3
Long request count: 13
Restarting short request
Short request count: 2
Long request count: 14
Short request count: 3
Long request c

In [35]:
file_path = 'bronze2_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(bronze2_matches, file)

In [36]:
bronze1_matches = getMatchesByRank("BRONZE", "I")

{'leagueId': 'bcd658c0-f9d6-4870-872b-977b559f8503', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'BRONZE', 'rank': 'I', 'summonerId': 'zKCFiXJzxJ6Um7p3RYjeFGljRTLzi6ZIK3woW1i1S23HCY7t', 'summonerName': 'IKiteFatPeople', 'leaguePoints': 85, 'wins': 3, 'losses': 2, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 2
Long request count: 6
Short request count: 3
Long request count: 7
Short request count: 4
Long request count: 8
Restarting short request
Short request count: 2
Long request count: 9
Short request count: 3
Long request count: 10
Short request count: 4
Long request count: 11
Restarting short request
Short request count: 2
Long request count: 12
Short request count: 3
Long request count: 13
Short request count: 4
Long request count: 14
Restarting short request
Short request count: 2
Long request count: 15
Short request co

In [37]:
file_path = 'bronze1_matches.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(bronze1_matches, file)

In [39]:
ranks = ["SILVER", "GOLD", "PLATINUM", "EMERALD", "DIAMOND"]
subranks = ["IV", "III", "II", "I"]

for rank in ranks:
    for subrank in subranks:
        matches = getMatchesByRank(rank, subrank)
        subrank_numeric = None
        match subrank:
            case "IV":
                 subrank_numeric = 4
            case "III":
                 subrank_numeric = 3
            case "II":
                 subrank_numeric = 2
            case "I":
                subrank_numeric = 1
            case _:
                subrank_numeric = None
        file_path = f"{rank.lower()}{subrank_numeric}_matches.pickle"
        with open(file_path, 'wb') as file:
            pickle.dump(matches, file)

{'leagueId': 'e6642c43-cc94-4466-9d5c-a4f93ce1076f', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'SILVER', 'rank': 'IV', 'summonerId': '-1cCHsgQZAfycZhdKjjKasjbAC8D_EZtzBpogUzmHxOQHtXT', 'summonerName': 'Trigno Kon', 'leaguePoints': 92, 'wins': 23, 'losses': 26, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 2
Long request count: 6
Short request count: 3
Long request count: 7
Restarting short request
Short request count: 2
Long request count: 8
Short request count: 3
Long request count: 9
Restarting short request
Short request count: 2
Long request count: 10
Short request count: 3
Long request count: 11
Restarting short request
Short request count: 2
Long request count: 12
Short request count: 3
Long request count: 13
Restarting short request
Short request count: 2
Long request count: 14
Short request count: 3
Long request co

C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\4035507561.py:84: UserWarning: Error: Response Code 403
  warnings.warn(f"Error: Response Code {resp.status_code}")


Short request count: 2
Long request count: 2


C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\4035507561.py:59: UserWarning: Error: Response Code 403
  warnings.warn(f"Error: Response Code {resp.status_code}")


Short request count: 3
Long request count: 3
Short request count: 4
Long request count: 4
Restarting short request
Short request count: 2
Long request count: 5
Short request count: 3
Long request count: 6
Short request count: 4
Long request count: 7
Restarting short request
Short request count: 2
Long request count: 8
Short request count: 3
Long request count: 9
Short request count: 4
Long request count: 10
Restarting short request
Short request count: 2
Long request count: 11
Short request count: 3
Long request count: 12
Short request count: 4
Long request count: 13
Restarting short request
Short request count: 2
Long request count: 14
Short request count: 3
Long request count: 15
Short request count: 4
Long request count: 16
Restarting short request
Short request count: 2
Long request count: 17
Short request count: 3
Long request count: 18
Short request count: 4
Long request count: 19
Restarting short request
Short request count: 2
Long request count: 20
Short request count: 3
Long r

C:\Users\dylan\AppData\Local\Temp\ipykernel_12608\4035507561.py:10: UserWarning: Error: Response Code 403
  warnings.warn(f"Error: Response Code {resp.status_code}")


********************FINISHED**********************
********************FINISHED**********************


In [54]:
diamond4_matches = None

with open("diamond4_matches.pickle", 'rb') as file:
    diamond4_matches = pickle.load(file)

In [56]:
print(len(diamond4_matches))

2050


In [59]:
subranks = ["III", "II", "I"]
rank = "DIAMOND"
for subrank in subranks:
    matches = getMatchesByRank(rank, subrank)
    subrank_numeric = None
    match subrank:
        case "IV":
             subrank_numeric = 4
        case "III":
             subrank_numeric = 3
        case "II":
             subrank_numeric = 2
        case "I":
            subrank_numeric = 1
        case _:
            subrank_numeric = None
    file_path = f"{rank.lower()}{subrank_numeric}_matches.pickle"
    with open(file_path, 'wb') as file:
        pickle.dump(matches, file)

{'leagueId': 'f703e96c-e6f8-4c59-8c32-aa098e159a94', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'DIAMOND', 'rank': 'III', 'summonerId': 'YIXmS-Ewsd49XMV5VvjXCbwFAT_gR-bd_YzmPAr3JqSYgFjB', 'summonerName': 'Chucktheman33', 'leaguePoints': 99, 'wins': 9, 'losses': 4, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': True}
Short request count: 2
Long request count: 4
Short request count: 3
Long request count: 5
Restarting short request
Short request count: 2
Long request count: 6
Short request count: 3
Long request count: 7
Restarting short request
Short request count: 2
Long request count: 8
Short request count: 3
Long request count: 9
Restarting short request
Short request count: 2
Long request count: 10
Short request count: 3
Long request count: 11
Restarting short request
Short request count: 2
Long request count: 12
Short request count: 3
Long request count: 13
Restarting short request
Short request count: 2
Long request count: 14
Short request count: 3
Long request 

In [63]:
class ApiRequestPipeline:
    """A pipeline class that facilitates chained API calls.

    Uses a series of callback functions defined in a list. 
    Each callback function should return the API request url/s,
    either as a single string or list of strings
    and accept the return of an API request as its parameter
    using the previous callback function's returned url/s.
    Two api rate limits can be added as tuples in the form (request_count, time).
    """
    
    def __init__(self, callback_list, rate_limit_1=None, rate_limit_2=None):
        self.callback_list = callback_list
        self.rate_limit_1 = rate_limit_1
        self.rate_limit_2 = rate_limit_2

    def make_requests(initial_request_url=None):
        

SyntaxError: incomplete input (3742492154.py, line 18)

In [64]:
help(ApiRequestPipeline)

Help on class ApiRequestPipeline in module __main__:

class ApiRequestPipeline(builtins.object)
 |  ApiRequestPipeline(callback_list, rate_limit_1, rate_limit_2)
 |
 |  A pipeline class that facilitates chained API calls.
 |
 |  Uses a series of callback functions defined in a list.
 |  Each callback function should return the API request url,
 |  and accept the return of an API request using the previous
 |  callback function's returned url. Two api rate limits can be
 |  added as tuples in the form (request_count, time).
 |
 |  Methods defined here:
 |
 |  __init__(self, callback_list, rate_limit_1, rate_limit_2)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



list